In [ ]:
%matplotlib notebook
%reload_ext autoreload
%autoreload 2

import datetime
from os import path, environ
from copy import deepcopy
from multiprocessing import Pool

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import (
    pad_sequence,
    pad_packed_sequence,
    pack_padded_sequence)
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from pytorchbridge import TorchEstimator

from utils import fit_composite_model, contiguous_sequences
from plotting import model_surface, plot_surface
from controller import GridSearchController, BinaryApproachController
# source file, see docs/5-dataset.md for info on field names
chiller_file = path.join(environ['DATADIR'],
                         'EngineeringScienceBuilding',
                         'Chillers.csv')
plot_path = path.join('..', 'docs', 'img')

In [ ]:
# Read pre-processed data:
# Pytorch uses float32 as default type for weights etc,
# so input data points are also read in the same type.
df = pd.read_csv(chiller_file, index_col='Time',
                 parse_dates=['Time'], dtype=np.float32)
df.dropna(inplace=True)

# Energy Model

In [ ]:
# Data
feature_cols = ('TempCondIn', 'TempCondOut', 'TempEvapIn',
                'TempEvapOut', 'TempAmbient', 'TempWetBulb',
                'FlowEvap')
target_cols = ('PowChi',)
# Normalizing data to have 0 mean and 1 variance
XEnergy, YEnergy = df.loc[:, feature_cols], df.loc[:,target_cols]
ScalerXEnergy, ScalerYEnergy = StandardScaler().fit(XEnergy), StandardScaler().fit(YEnergy)
XEnergy, YEnergy = ScalerXEnergy.transform(XEnergy), np.squeeze(ScalerYEnergy.transform(YEnergy))
XEnergytrain, XEnergytest, YEnergytrain, YEnergytest = train_test_split(XEnergy, YEnergy, test_size=0.1)


norm_energy_var = lambda i, v: ScalerXEnergy.mean_[i] + np.sqrt(ScalerX.var_[i]) * v
norm_energy = lambda v: ScalerYEnergy.inverse_transform(v)

ENERGY_MODELS = {}

## Multi-layer perceptron

In [ ]:
# Learn model
regressor = MLPRegressor(hidden_layer_sizes=(20,20), 
                         learning_rate_init=5e-4,
                         verbose=False)
est = Pipeline([('regressor', regressor)])

est.fit(XEnergytrain, YEnergytrain)
print(est.score(XEnergytest, YEnergytest))
ENERGY_MODELS['SingleMLP'] = est

In [ ]:
# Plot prediction vs actual across day or timestamp

# time = datetime.time(12,0,0)
time = datetime.date(2018,6,1)

if isinstance(time, datetime.time):
    select = df.index.time == time
    tseries = df.index.date[select]
elif isinstance(time, datetime.date):
    select = df.index.date == time
    tseries = df.index.time[select]
Xfiltered = X[select]
Yfiltered = Y[select]

plt.scatter(np.arange(len(tseries)), ScalerYEnergy.inverse_transform(Yfiltered), c='r', label='Historic')
plt.scatter(np.arange(len(tseries)), ScalerYEnergy.inverse_transform(est.predict(Xfiltered)), c='g', label='Predicted')
plt.title('Energy Model ' + time.isoformat())
plt.ylabel('Energy')
plt.xlabel('Date/Time')
plt.legend();

In [ ]:
# Plot model predictions when varying a field
var1= 'TempCondOut'
var_idx = feature_cols.index(var1)

_, y, z = model_surface(est, Xfiltered, (var1idx,), ((-0.5, 0.5),), (10,))
ax = plot_surface(tseries,
                  norm_energy_var(var1idx, y),
                  ScalerY.inverse_transform(z),
                  alpha=0.75, cmap=plt.get_cmap('coolwarm'))
ax.set_xlabel('Date/Time', labelpad=20)
ax.set_ylabel(var1)
ax.set_zlabel('Energy')
plt.title('Energy Extrapolation ' + time.isoformat());

# Evaporative Cooling Model

Modelling the relationship between the cooling done by the cooling tower and the environmental, system, and control inputs.

$$
\begin{align*}
T(t) &= T(0) e^{-\frac{k T_a v_f R}{T_w c_m m} t}
\end{align*}
$$

Where:

* $T(0)$ is the warm entering water temperature `TempCondOut`,
* $T(t)$ is the exiting cool water temperature `TempCondIn`,
* $T_a$ is ambient air temperature `TempAmbient`,
* $v_f$ is avarage fan speed `0.5 * (PerFreqFanA + PerFreqFanB)`,
* $R$ is solar irradiance - constant if assuming shade.
* $T_w$ is wet-bulb temperature `TempWetBulb`,
* $c_m$ is specific mass heat capacity of water,
* $m$ is the mass of water being cooled - constant if assuming steady flow rate.

The model predicts $T(t)$ from all other factors.

In [ ]:
# Data
control_cols = ('PerFreqFanA', 'PerFreqFanB')
feature_cols = ('TempCondOut', 'TempAmbient', 'TempWetBulb')
target_cols = ('TempCondIn',)
# Combining 2 fan speed controls into a single variable (averaged)
X = pd.concat((df.loc[:, control_cols].mean(axis=1), 
               df.loc[:, feature_cols]), axis=1)
Y = df.loc[:,target_cols]
# Normalizing data to have 0 mean and 1 variance
ScalerX, ScalerY = StandardScaler().fit(X), StandardScaler().fit(Y)
X, Y = ScalerX.transform(X), np.squeeze(ScalerY.transform(Y))
# generating training/testing sets
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.1)

# Convert feature variables or target to normal or standardized form:
# i - is the index of the variable in the feature array
# v - is an array of values to transform
norm_var = lambda i, v: ScalerX.mean_[i] + np.sqrt(ScalerX.var_[i]) * v
std_var = lambda i, v: (v - ScalerX.mean_[i]) / np.sqrt(ScalerX.var_[i])
norm_temp = lambda v: ScalerY.inverse_transform([v])[0]
std_temp = lambda v: ScalerY.transform([v])[0]

EVAP_MODELS = {}

## Multi-Layer Perceptron

The motivation for an MLP model is the assumption that inputs manifest instantaneously as outputs with no delay. This is a simplification as water takes some time to cycle through the cooling tower.

The inputs are chosen assuming steady flow rate, constant irradiance.

The control variables are `PerFreqFanA` and `PerFreqFanB` which usually track each other. They are distributed bi-modally around 100% and 0% power with a minority of samples falling in between. A concern is that the model may learn to treat control inputs as constant. Two approaches are chosen:

* A single model is learned over all data,
* Samples are clustered by control=0, control >= 0.95, and 0 < control  < 0.95. A separate model is learned for each cluster.

### Single MLP

In [ ]:
# Learn model
regressor = MLPRegressor(hidden_layer_sizes=(20,20), 
                         learning_rate_init=5e-4,
                         verbose=False) # regression model
est = Pipeline([('regressor', regressor)])

est.fit(Xtrain, Ytrain)
EVAP_MODELS['SingleMLP'] = est

In [ ]:
# Get predictions
var_idx = 0
nvariations = 11   # range of fan speed percentages to explore
# time = datetime.time(4,0,0)
time = datetime.date(2018,6,1)

if isinstance(time, datetime.time):
    select = df.index.time == time
    tseries = df.index.date[select]
elif isinstance(time, datetime.date):
    select = df.index.date == time
    tseries = df.index.time[select]
Xfiltered = X[select]
Yfiltered = Y[select]
vary_range = (-ScalerX.mean_[var_idx]/np.sqrt(ScalerX.var_[var_idx]),
              (1-ScalerX.mean_[var_idx])/np.sqrt(ScalerX.var_[var_idx]))
time_coord, fan_coord, temp_coord = model_surface(est, Xfiltered,
                        vary_idx=(var_idx,), vary_range=(vary_range,), vary_num=(10,))

In [ ]:
# Plot optimal control

min_fan_idx = np.argmin(temp_coord, axis=1).astype(int)

ax = plot_surface(tseries,
                  norm_var(var_idx, fan_coord),
                  norm_temp(temp_coord),
                  alpha=0.75, cmap=plt.get_cmap('coolwarm'))
ax.scatter(time_coord[:,0],
        norm_var(var_idx, Xfiltered[:, 0]),
        norm_temp(Yfiltered),
       c='red', label='Historic fan speed')
ax.scatter(time_coord[:,0],
           norm_var(0, fan_coord[np.arange(len(fan_coord)).astype(int), min_fan_idx]),
           norm_temp(temp_coord[np.arange(len(temp_coord)).astype(int), min_fan_idx]),
          c='g', label='Optimal fan speed')

ax.set_xlabel('Time')
ax.set_ylabel('Fan speed %')
ax.set_zlabel('Temperature')
ax.legend()

In [ ]:
print('Score on all control:\t{:.4f}'.format(est.score(Xtest, Ytest)))

## RNN

The MLP model assumes no delay between input variables and output. This is a simplifying assumption. Water takes some time to travel through the cooling tower. So the temperature of the water coming out is a function of a history of the tower's prior states *and* the current inputs.

In [ ]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.1, shuffle=False)
# indices of contiguous sequences in df
seqIdxTrain = contiguous_sequences(Xtrain.index, pd.Timedelta('5T'))
seqIdxTest = contiguous_sequences(Xtest.index, pd.Timedelta('5T'))
print(f'{len(seqIdxTrain)} sequences in train set')
print(f'{len(seqIdxTest)} sequences in test set')

In [ ]:
# Generating sequences for training.
# seqX: SeqLength x N x 5 features, seqY: SeqLength x N
# Where SeqLength is the length of the longest sequence.
# The sequences are sorted by length and 0-padded.
seqX = list(map(lambda idx: torch.tensor(X.loc[idx].values), seqIdxTrain))
seqX = sorted(seqX, key=len)
seqX = pad_sequence(seqX, batch_first=False)

seqY = list(map(lambda idx: torch.tensor(Y.loc[idx].values), seqIdxTrain))
seqY = sorted(seqY, key=len)
seqY = pad_sequence(seqY, batch_first=False)

In [ ]:
# creating an LSTM
# The model accepts data in the form:
#     [SequenceLength (L) x BatchSize (N) x Features (F)]
# This is the *default* form RNNs accept data in, and is explicitly
# passed on using the argument `batch_first=False`.
class Model(nn.Module):
    
    def __init__(self, features=5, batch_first=False):
        super().__init__()
        self.batch_first = batch_first
        self.features = features
        self.fc1 = nn.Linear(self.features, 10)
        self.fc2 = nn.Linear(10, 10)
        self.rnn1 = nn.RNN(input_size=10, hidden_size=5,
                           nonlinearity='relu', batch_first=batch_first)
        self.rnn2 = nn.RNN(input_size=10, hidden_size=5,
                           nonlinearity='relu', batch_first=batch_first)
        self.fc3 = nn.Linear(10, 1)
    
    
    def forward(self, u):
        # x' obtained from u
        x_ = F.relu(self.fc1(u))       # L, N, 10
        x_ = F.relu(self.fc2(x_))      # L, N, 10
        # x obtained from x'
        x1, _ = self.rnn1(x_)          # L, N, 5
        x2, _ = self.rnn2(x_)          # L, N, 5
        x = torch.cat((x1, x2), dim=2) # L, N, 10
        # y obtained from x
        y = self.fc3(x)                # L, N, 1
        return y.squeeze(dim=2)        # L, N

m = Model()

o = m(torch.rand(10,2,5))
print(o.size())

In [ ]:
tx, ty = torch.rand(20,10,5), torch.rand(20,10)
m = Model()
regressor = TorchEstimator(module=m, optimizer=torch.optim.Adam(m.parameters()),
                    loss=nn.MSELoss(), epochs=4, batch_size=5, verbose=True,
                    cuda=False)
est = Pipeline([('regressor', regressor)])
est.fit(tx, ty)
EVAP_MODEL['RNN'] = est

# Optimizing fan speed for temperature

In [ ]:
# time = datetime.time(4,0,0)
time = datetime.date(2018,6,1)
if isinstance(time, datetime.time):
    select = df.index.time == time
    tseries = df.index.date[select]
elif isinstance(time, datetime.date):
    select = df.index.date == time
    tseries = df.index.time[select]
Xfiltered = X[select]
Yfiltered = Y[select]
XEnergyfiltered = XEnergy[select]
YEnergyfiltered = YEnergy[select]

vary_range = (-ScalerX.mean_[0]/np.sqrt(ScalerX.var_[0]),
              (1-ScalerX.mean_[0])/np.sqrt(ScalerX.var_[0]))
margin = (std_temp([3]) - std_temp([0]))[0]

est = EVAP_MODELS['SingleMLP']
est_energy = ENERGY_MODELS['SingleMLP']
gridc = GridSearchController(est, vary_range, 10, 0)
binaryc = BinaryApproachController(est, vary_range, 10, 0, margin)

In [ ]:
# Comparison of control over a contiguous time period
Xdata = Xfiltered

baseline = np.squeeze(std_temp(norm_var(3, Xdata[:, 3])))

grid_fan = norm_var(0, gridc.predict(Xdata))
binary_fan = norm_var(0, binaryc.predict(Xdata, baseline))

gridXdata, binaryXdata = np.copy(Xdata), np.copy(Xdata)
gridXdata[:, 0], binaryXdata[:, 0] = grid_fan, binary_fan

plt.figure()
sgc = plt.plot(np.arange(len(grid_fan)), grid_fan, label='Grid Controller')
sbc = plt.plot(np.arange(len(grid_fan)), binary_fan, label='Binary Controller')
plt.sca(plt.gca().twinx())
sgt = plt.plot(np.arange(len(grid_fan)), norm_temp(est.predict(gridXdata)), label='Grid Temps')
sbt = plt.plot(np.arange(len(grid_fan)), norm_temp(est.predict(binaryXdata)), label='Binary Temps')
plots = sgc + sbc + sgt + sbt
labels = [p.get_label() for p in plots]
plt.legend(plots, labels)
plt.show()

In [ ]:
# Energy saving plots
Xdata = XEnergyfiltered
grid_temp, binary_temp = est.predict(gridXdata), est.predict(binaryXdata)
gridXdata, binaryXdata = np.copy(Xdata), np.copy(Xdata)
gridXdata[:, 0], binaryXdata[:, 0] = grid_temp, binary_temp

grid_energy, binary_energy = est_energy.predict(gridXdata), est_energy.predict(binaryXdata)

plt.figure()
sgc = plt.scatter(np.arange(len(grid_energy)), grid_energy, label='Grid Controller')
sbc = plt.scatter(np.arange(len(grid_energy)), binary_energy, label='Binary Controller')
plt.legend()
plt.show()

In [ ]:
# Comparison of control over a random sample
Xdata = Xtest

baseline = np.squeeze(std_temp(norm_var(3, Xdata[:, 3])))

grid_fan = gridc.predict(Xdata)
binary_fan = binaryc.predict(Xdata, baseline)

gridXdata, binaryXdata = np.copy(Xdata), np.copy(Xdata)
gridXdata[:, 0], binaryXdata[:, 0] = grid_fan, binary_fan

plt.figure()
plt.subplot(121)
plt.hist(norm_var(0, grid_fan), label='Grid Controller', histtype='step', density=True)
plt.hist(norm_var(0, binary_fan), label='Binary Controller', histtype='step', density=True)
plt.legend()
plt.subplot(122)
plt.hist(norm_temp(est.predict(gridXdata)), label='Grid Temps', histtype='step')
plt.hist(norm_temp(est.predict(binaryXdata)), label='Binary Temps', histtype='step')
plt.legend()
plt.show()

In [ ]:
# Energy saving plots
Xdata = XEnergytest
grid_temp, binary_temp = est.predict(gridXdata), est.predict(binaryXdata)
gridXEnergydata, binaryXEnergydata = np.copy(Xdata), np.copy(Xdata)
gridXEnergydata[:, 0], binaryXEnergydata[:, 0] = grid_temp, binary_temp

grid_energy, binary_energy = est_energy.predict(gridXEnergydata), est_energy.predict(binaryXEnergydata)

plt.figure()
sgc = plt.hist(norm_energy(grid_energy), label='Grid Controller', histtype='step')
sbc = plt.hist(norm_energy(binary_energy), label='Binary Controller', histtype='step')
plt.legend()
plt.show()